In [1]:
import os

In [2]:
%pwd

'/Users/mark42/Documents/ML-Pipeline/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/mark42/Documents/ML-Pipeline'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class OptimizingConfig:
    root_dir: Path
    optimized_model_path: Path
    model_path:Path
    train_data_loader_path:Path
    test_data_loader_path:Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_optimizing_config(self) -> OptimizingConfig:
        config = self.config.optimizing
        create_model = self.config.create_model
        working_with_data = self.config.working_with_data

        create_directories([config.root_dir])

        optimizing_config = OptimizingConfig(
            root_dir=Path(config.root_dir),
            optimized_model_path=Path(config.optimized_model_path),
            model_path=Path(create_model.model_path),
            train_data_loader_path=Path(working_with_data.train_data_loader_path),
            test_data_loader_path=Path(working_with_data.test_data_loader_path),
        )

        return optimizing_config

In [8]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [9]:
class Optimizing:
    def __init__(self, config: OptimizingConfig):
        self.config = config

    def get_train_data_loaders(self):
        self.train_data_loader = torch.load(self.config.train_data_loader_path)

    def get_test_data_loaders(self):
        self.test_data_loader = torch.load(self.config.test_data_loader_path)

    def train(dataloader, model, loss_fn, optimizer, device):
        size = len(dataloader.dataset)
        model.train()
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            # Compute prediction error
            pred = model(X)
            loss = loss_fn(pred, y)

            # Backpropagation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if batch % 100 == 0:
                loss, current = loss.item(), (batch + 1) * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    def test(dataloader, model, loss_fn, device):
        size = len(dataloader.dataset)
        num_batches = len(dataloader)
        model.eval()
        test_loss, correct = 0, 0
        with torch.no_grad():
            for X, y in dataloader:
                X, y = X.to(device), y.to(device)
                pred = model(X)
                test_loss += loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [10]:
try:
    config = ConfigurationManager()
    optimizing_config = config.get_optimizing_config()
    
    optimizing = Optimizing(config=optimizing_config)

    device = (
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )
    print(f"Using {device} device")

    model = CreateModel().to(device)
    model.load_state_dict(torch.load(optimizing_config.model_path))
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    epochs = 5
    train_dataloader = optimizing.get_train_data_loaders()
    test_dataloader = optimizing.get_test_data_loaders()
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        optimizing.train(train_dataloader, model, loss_fn, optimizer)
        optimizing.test(test_dataloader, model, loss_fn)
    print("Done!")
    
except Exception as e:
    raise e

    

[2024-08-12 19:01:02,204: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-12 19:01:02,207: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-12 19:01:02,207: INFO: common: created directory at: artifacts]
[2024-08-12 19:01:02,208: INFO: common: created directory at: artifacts/optimizing]
Using mps device


NameError: name 'CreateModel' is not defined